In [8]:
import pandas as pd
import requests
from tqdm import tqdm
from typing import Optional

In [6]:
class ApiClient:
    def __init__(
            self, 
            username: Optional[str] = None, 
            password: Optional[str] = None,
            base_url: str = 'http://127.0.0.1:8000'
        ):
        self.base_url = base_url

        if username is not None:
            self.login(username, password)

    def login(self, username: str, password: str):
        url = f'{self.base_url}/authentication/login?username={username}&password={password}'
        r = requests.post(url)
        r.raise_for_status()
        r_json = r.json()
        self.auth_header = {"Authorization": f"{r_json['token_type'].title()} {r_json['access_token']}"}
    
    def get_db_table_schema(self, table_name: str):
        r = requests.get(f'{self.base_url}/frictionless/db-schema/{table_name}', headers=self.auth_header)
        r.raise_for_status()
        return r.json()
    
    def post_data_package_fp(self, data_package_fp: str):
        r = requests.post(f'{self.base_url}/frictionless/data-packages?data_package_fp={data_package_fp}', headers=self.auth_header)
        r.raise_for_status()
        return r.json()

In [7]:
api_client = ApiClient('tester', 'password')

api_client

<br>

### Uploading Data Packages

We'll start by uploading the data packages

In [4]:
fps = [
    '../data/linked-datapackages/plant-locations',
    '../data/linked-datapackages/wind-farms',
    '../data/linked-datapackages/global-power-plant-database',
    '../data/linked-datapackages/nuclear-power-plants',
    '../data/linked-datapackages/cfd-contract-portfolio-status',
    '../data/linked-datapackages/renewable-energy-planning-database',
    '../data/linked-datapackages/bmu-fuel-types',
    '../data/linked-datapackages/jrc-hydro-power-plants-database',

    # the following require handling of time-series indexes
    # '../data/linked-datapackages/cfd-strike-prices',
    # '../data/linked-datapackages/annual-output',
    # '../data/linked-datapackages/load-factors',
    # '../data/linked-datapackages/carbon-intensity',
    # '../data/linked-datapackages/verified-emissions',
    # '../data/linked-datapackages/capture-prices'
]

for fp in tqdm(fps):
    api_client.post_data_package_fp(fp.lstrip('../') + '/datapackage.json')

100%|██████████| 8/8 [00:13<00:00,  1.71s/it]


<br>

We can then retrieve their DB table schemas using the API client

In [5]:
api_client.get_db_table_schema('dict__source_detailed_bmu_fuel_types')

{'title': 'DetailedBmuFuelTypesTable',
 'type': 'object',
 'properties': {'ngc_bmu_id': {'title': 'NGC BMU ID',
   'description': 'The Balancing Mechanism Unit identifier used by the National Grid Company',
   'type': 'string'},
  'fuel_type': {'title': 'Fuel Type',
   'description': 'The fuel type consumed by the specified BMU',
   'type': 'string'},
  'comments': {'title': 'Comments', 'type': 'string'}}}

In [ ]:
# add the IDs directly to the DB
# ensure that assets are loading their linked data

In [10]:
df_ids = pd.read_csv('../data/dictionary/ids.csv')

df_ids.head(3)

,dictionary_id,gppd_idnr,esail_id,name,sett_bmu_id,ngc_bmu_id,4c_offshore_id,windpowernet_id,wikidata_id,wikipedia_id,power_technology_id,eutl_id,eic_id,cfd_id,jrc_id,iaea_id,old_repd_id,new_repd_id,crown_estate_id
0,10000,NaN,MARK,Rothes Bio-Plant CHP,"E_MARK-1, E_MARK-2","MARK-1, MARK-2",NaN,NaN,NaN,NaN,NaN,NaN,48W000000MARK-1D,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,"GBR1000377, GBR1000369",DIDC,Didcot,"T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3, T_DIDC1G, ...","DIDC1, DIDC2, DIDC4, DIDC3, DIDC1G, DIDC2G, DI...",NaN,NaN,Q3298465,Didcot_power_stations,NaN,97165,"48W00000DIDC01G1, 48W00000DIDC02GZ, 48W00000DI...",NaN,NaN,NaN,NaN,NaN,NaN
2,10002,"GBR1000374, GBR1000375",ABTH,Aberthaw B,"T_ABTH7, T_ABTH8, T_ABTH9, T_ABTH7G, T_ABTH8G,...","ABTH7, ABTH8, ABTH9, ABTH7G, ABTH8G, ABTH9G",NaN,NaN,Q4667192,Aberthaw_power_stations,NaN,97175,"48W0000000ABTH7Y, 48W0000000ABTH8W, 48W0000000...",NaN,NaN,NaN,NaN,NaN,NaN
